<a href="https://colab.research.google.com/github/MicroprocessorX069/Collaborative-Filtering-for-medical-history/blob/master/CF_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df=pd.read_csv("dummy.csv",header=None)

In [0]:
import math
#adjacent elements of original dataset added here in df_adj
df_adj=pd.DataFrame()
for x in range(len(df)):
  y=0
  while y+1<len(df.iloc[x]) and not(pd.isna(df.iloc[x,y+1])):
    temp=df.iloc[x,y]
    temp2=df.iloc[x,y+1]
    df2 = pd.DataFrame([[temp,temp2]])
    df_adj=df_adj.append(df2)
    y+=1
#df_adj=df_adj.reset_index() # to make the indices of rows as 0,1,2,3...

In [0]:
df_adj=pd.get_dummies(df_adj)
processedData=df_adj[['0_a',	'0_b',	'0_c',	'0_d',	'0_e'	]]
processedLabel=df_adj[['1_a',	'1_b',	'1_c',	'1_d',	'1_e'	]]

In [0]:
#@title Parameters for the model { run: "auto" } 

import tensorflow as tf
#run:auto command makes the code block run in real time. Whenever the user amends the input, the block runs automatically for the new set of input.
no_of_layers= 3 #@param {type:"slider", min:0, max:10, step:1}
# A is the list of nodes_in_each_hidden_layer for each of the user defined hidden layers. Eg. A=[128,256,64] implies 3 hidden layers with 128 nodes, 256 nodes and 64 nodes respectively starting from the input layer to the output layer.
A=[]
# No. of nodes in hidden layer 1
nodes_in_layer1 = 64 #@param {type:"integer"}
A.append(nodes_in_layer1)
#dropout_rate = 0.1 #@param {min:0.000,max:1.000}

nodes_in_layer2 = 64 #@param {type:"integer"}
A.append(nodes_in_layer2)
nodes_in_layer3 = 64 #@param {type:"integer"}
A.append(nodes_in_layer3)
validation_splitt=0.2 #@param{type:"slider",min:0.000,max:1.000,step:0.05}
Activation_function = tf.nn.crelu #@param ["relu", "relu6", "crelu", "elu", "selu", "softplus", "softsign", "dropout", "bias_add", "sigmoid", "tanh"]
loss_function = 'kullback_leibler_divergence' #@param ["categorical_crossentropy", "sparse_categorical_crossentropy", "binary_crossentropy","kullback_leibler_divergence","poisson","cosine_proximity"]
optimizer_used = 'SGD' #@param ["SGD", "RMSprop", "Adagrad","Adadelta","Adam","Adamax","Nadam","TFOptimizer"]

In [65]:
#@title Parametres for the model { run: "auto" }

#no_epochs is the integer input for total no. of model parsing -
#from input to generating output and back to updating the weights -
#according to the error for all training examples.

no_epochs=200 #@param{type:"slider",min:0,max:10000,step:200}
 
import time
start=time.time()

#Libraries for current codeblock
import tensorflow as tf
from tensorflow import nn
nodes_in_output_layer=5
input_size=5

model=tf.keras.models.Sequential()

#Various types of layers in keras: Dense, Flatten, Input, Permute, Reshape.[3]
#Dense implements the operation: output = activation(dot(input, kernel) + bias)[3]

#Adding input layer with size of input i.e. 10 in this case. 
#The first parameter for function Dense() is No. of nodes. in the layer. model.add appends the newly created hidden layer to the model.

model.add(tf.keras.layers.Dense(A[0],activation=Activation_function,input_dim=input_size))
for layer_iterator in range(1,len(A)-1):
  model.add(tf.keras.layers.Dense(A[layer_iterator+1], activation=Activation_function))
#Creating dropout layer
#model.add(tf.keras.layers.Dropout(rate=dropout_rate))
# Creating the output layer with 4 nodes. Since Output layer 
model.add(tf.keras.layers.Dense(nodes_in_output_layer, activation=tf.nn.softmax))

# Parameters for model creation i.e. optimizers, loss function
                                                                                # metric: Accuracy. Accuracy is the measure of correctness of the model
                                                                                # - , basically rate to which it classifies the inputs correctly.
                                                                                # Validation Accuracy: Percentage of correctly classifying on input chosen from the train dataset itself.
                                                                                # Testing Accuracy:  Percentage of correctly classifying on input from a new dataset following the same problem as the train dataset.

                                                                                # Optimizer: Algorithm to minimize the error/loss by updating weights to improve the training of the model. e.g.
model.compile(optimizer=optimizer_used, loss=loss_function, metrics=['accuracy'])
model.summary()
history=model.fit(processedData, processedLabel, epochs= no_epochs, validation_split= validation_splitt)

end=time.time()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               384       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 9,285
Trainable params: 9,285
Non-trainable params: 0
_________________________________________________________________
Train on 27 samples, validate on 7 samples
Epoch 1/200
27/27 [==============================] - 1s 24ms/step - loss: 1.6042 - acc: 0.1852 - val_loss: 1.6255 - val_acc: 0.1429
Epoch 2/200
27/27 [==============================] - 0s 113us/step - loss: 1.6004 - acc: 0.1852 - val_loss: 1.6273 - val_acc: 0.1429
Epoch 3/200
27/27 [==============================] - 0s 154us/step - loss: 1.5968 - acc: 0.1852 

In [58]:
df_adj

,0_a,0_b,0_c,0_d,0_e,1_a,1_b,1_c,1_d,1_e
0,1,0,0,0,0,0,1,0,0,0
0,0,1,0,0,0,0,0,1,0,0
0,0,0,1,0,0,1,0,0,0,0
0,1,0,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,1,0,0,0
0,0,1,0,0,0,0,0,1,0,0
0,0,0,1,0,0,0,0,1,0,0
0,0,0,1,0,0,0,1,0,0,0
0,1,0,0,0,0,0,1,0,0,0
0,0,1,0,0,0,0,1,0,0,0
